In [1]:
import pandas as pd
import json
from datetime import datetime, timedelta

In [2]:
data = pd.read_csv('data.csv')

In [3]:
data.head()

,id,application_date,contracts
0,2925210.0,2024-02-12 19:22:46.652000+00:00,NaN
1,2925211.0,2024-02-12 19:24:29.135000+00:00,"[{""contract_id"": 522530, ""bank"": ""003"", ""summa..."
2,2925212.0,2024-02-12 19:24:41.493000+00:00,NaN
3,2925213.0,2024-02-12 19:24:29.135000+00:00,"[{""contract_id"": 522530, ""bank"": ""003"", ""summa..."
4,2925214.0,2024-02-12 19:24:56.857000+00:00,NaN


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 3 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   id                1000 non-null   float64
 1   application_date  1000 non-null   object 
 2   contracts         495 non-null    object 
dtypes: float64(1), object(2)
memory usage: 23.6+ KB


In [5]:
data['id'] = data['id'].astype('Int64')

In [6]:
data['application_date'] = pd.to_datetime(data['application_date'],format='mixed')

In [7]:
data['contracts'] = data['contracts'].apply(lambda x: json.loads(x) if pd.notnull(x) else [])

In [8]:
data = data.explode('contracts')
data = data.reset_index(drop=True)

In [9]:
contracts = pd.json_normalize(data['contracts'])

In [10]:
data = pd.concat([data.drop(columns=['contracts']), contracts], axis=1)

In [14]:
data.head()

,id,application_date,contract_id,bank,summa,loan_summa,claim_date,claim_id,contract_date
0,2925210,2024-02-12 19:22:46.652,NaN,NaN,NaN,NaN,NaT,NaN,NaT
1,2925211,2024-02-12 19:24:29.135,522530,003,500000000.0,0.0,2020-02-13,609965,2020-02-17
2,2925211,2024-02-12 19:24:29.135,,014,NaN,NaN,2020-08-28,F00013731,NaT
3,2925211,2024-02-12 19:24:29.135,,014,NaN,NaN,2020-10-08,F00021301,NaT
4,2925211,2024-02-12 19:24:29.135,,014,NaN,NaN,2020-11-25,F00037907,NaT


In [15]:
def convert_data_types(df):
    df['application_date'] = pd.to_datetime(df['application_date'], errors='coerce').dt.tz_localize(None)
    df['claim_date'] = pd.to_datetime(df['claim_date'], format='%d.%m.%Y', errors='coerce').dt.tz_localize(None)
    df['contract_date'] = pd.to_datetime(df['contract_date'], format='%d.%m.%Y', errors='coerce').dt.tz_localize(None)
    df['summa'] = pd.to_numeric(df['summa'], errors='coerce')
    df['loan_summa'] = pd.to_numeric(df['loan_summa'], errors='coerce')
    return df

In [16]:
data = convert_data_types(data)
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10069 entries, 0 to 10068
Data columns (total 9 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   id                10069 non-null  Int64         
 1   application_date  10069 non-null  datetime64[ns]
 2   contract_id       9220 non-null   object        
 3   bank              9078 non-null   object        
 4   summa             1894 non-null   float64       
 5   loan_summa        1714 non-null   float64       
 6   claim_date        9220 non-null   datetime64[ns]
 7   claim_id          9220 non-null   object        
 8   contract_date     1894 non-null   datetime64[ns]
dtypes: Int64(1), datetime64[ns](3), float64(2), object(3)
memory usage: 717.9+ KB


In [19]:
data.head()

,id,application_date,contract_id,bank,summa,loan_summa,claim_date,claim_id,contract_date
0,2925210,2024-02-12 19:22:46.652,NaN,NaN,NaN,NaN,NaT,NaN,NaT
1,2925211,2024-02-12 19:24:29.135,522530,003,500000000.0,0.0,2020-02-13,609965,2020-02-17
2,2925211,2024-02-12 19:24:29.135,,014,NaN,NaN,2020-08-28,F00013731,NaT
3,2925211,2024-02-12 19:24:29.135,,014,NaN,NaN,2020-10-08,F00021301,NaT
4,2925211,2024-02-12 19:24:29.135,,014,NaN,NaN,2020-11-25,F00037907,NaT
